In [29]:
#python3

# start time oct 24 - 21:10 - 01:06
# read dqn paper nov 19 - 22:23 - 22:55
# add fog nov 25 - 23:15 - 23:33
# more on moves nov 26 - 8:00 - 9:00 | 9:49 - 12:21

import numpy as np


class Map_thing():
  id = 0
  def __init__(self, t_char, mohs):
    self.t_char = t_char
    self.mohs = mohs
    self.id = Map_thing.id
    self.row = None
    self.col = None
    Map_thing.id += 1

  def print_self(self):
    return self.t_char

  def get_mohs(self):
    return self.mohs

  def __repr__(self):
    return self.t_char

  def __eq__(self, other):
    return other != None and self.t_char == other.t_char and self.id == other.id

  def set_location(self, row, col):
    self.row = row
    self.col = col

class Controller():
  def __init__(self, move_func=None, responder=None, creature=None, looker=None):
    self.move_func = move_func if move_func is not None else self._random
    self.responder = responder if responder is not None else print
    self.looker = looker if looker is not None else print
    self.creature = creature

  def _random(self):
    x = np.random.randint(0, 4)
    if x == 0:
      return 'north'
    elif x == 1:
      return 'south'
    elif x == 2:
      return 'east'
    elif x == 3:
      return 'west'
    return x

  def look(self, look_info):
    self.looker(look_info)

  def canMove(self):
    if self.creature != None:
      return self.creature.canMove()
    return False
  
  def getMove(self):
    return self.move_func()
  
  def sendResponse(self, message):
    self.responder(message)

  def setCreature(self, creature):
    self.creature = creature

  def getCreature(self):
    return self.creature

class SimpleController(Controller):
  def __init__(self, creature=None, prey=None, responder=None, looker=None):
    super().__init__(creature=creature, move_func=self._getMove, responder=responder, looker=looker)
    self.prey = prey
    if (self.creature == None or self.prey == None):
      self.move_func = super()._random
    self.last = ['', '']

  def _try_row(self, row_diff):
    result = ''
    if row_diff > 0:
        result = 'north'
    elif row_diff < 0:
        result = 'south'
    return result

  def _try_col(self, col_diff):
    result = ''
    if col_diff > 0:
        result = 'west'
    elif col_diff < 0:
        result = 'east'
    return result

  def sendResponse(self, message):
    self.last[1] = message
    super().sendResponse(message)

  def _getMove(self, fudge=0):
    row_diff = (self.creature.row - self.prey.row) + fudge
    col_diff = self.creature.col - self.prey.col

    result = ''
    # print(self.creature.id, row_diff, col_diff)
    if abs(row_diff) > abs(col_diff):
      result = self._try_row(row_diff)
      if result == '':
        result = self._try_col(col_diff)
    elif abs(row_diff) < abs(col_diff):
      result = self._try_col(col_diff)
      if result == '':
        result = self._try_row(row_diff)
    else:
      result = self._getMove(np.random.rand() - .5) # make even chance

    if result == '':
      result = self._random()      

    super().sendResponse(str(result))
    self.last = [result, '']
    return result

class ConsoleController(Controller):
  def __init__(self, creature=None, responder=None):
    super().__init__(creature=creature, move_func=self._getMove, responder=responder)
  
  def _getMove(self):
    result = input("Your move >")
    return result

class Creature(Map_thing):
  def __init__(self, t_char, mohs, num_moves=1, wait_moves=0, drop_thing=None):
    super().__init__(t_char, mohs)
    self.num_moves = num_moves
    self.max_wait = wait_moves
    self.wait_count = self.max_wait
    self.drop_thing = drop_thing

  def canMove(self):
    if self.wait_count <= 0:
      self.wait_count = self.max_wait
      return True
    self.wait_count -= 1
    return False

  def getNumMoves(self):
    return self.num_moves

  def getMove(self):
    return self.controller.getMove()

  def sendResponse(self, message):
    self.controller.sendResponse(message)

  def get_drop(self):
    return self.drop_thing

class Map:
  def __init__(self, width, height, density=20):
    self.width = width
    self.height = height
    self.density = density

    self._wall = Map_thing("█",10)
    self._space = Map_thing(' ',0)
    self._fog = Map_thing('*', 10)

    self._map = []
    self.re_init_map(self.density)

  def re_init_map(self, density=None):
    den = density if density is not None else self.density
    self._map = []
    for row in range(self.height):
      the_row = []
      if row == 0 or row == self.height - 1:
        the_row = [[self._wall] for _i in range(self.width)]
      else:
        the_row = [[self._wall]]
        the_row.extend([[self._wall] if np.random.randint(100) <= den else [self._space] for i in range(self.height - 2)])
        the_row.append([self._wall])
      self._map.append(the_row)

  def place(self, map_thing, to_row, to_col):
    worked = True
    if to_row <= 0 or to_row >= self.height:
      worked = False
    if to_col <= 0 or to_col >= self.width:
      worked = False
    if map_thing.mohs <= self._map[to_row][to_col][-1].mohs:
      worked = False
    if not worked:
      return 'Bad Move'
    self._map[to_row][to_col].append(map_thing)
    map_thing.set_location(to_row, to_col)
    return 'Moved'

  def rand_place(self, map_thing, tries=10):
    result = ''
    for _x in range(tries):
      result = self.place(map_thing, np.random.randint(1, self.height -1), np.random.randint(1, self.width - 1))
      if result == 'Moved':
        return result
    return result


  def move(self, creature, from_row, from_col, move):
    to_row = from_row
    to_col = from_col

    if move == 'north' or move == 0:
      to_row -= 1
    elif move == 'south' or move == 1:
      to_row += 1
    elif move == 'west' or move == 2:
      to_col -= 1
    elif move == 'east' or move == 3:
      to_col += 1
    else:
      return 'Unknown Move'
    
    place_result = self.place(creature, to_row, to_col)
    if place_result == 'Moved':
      self._map[from_row][from_col].remove(creature)
      if creature.get_drop() != None:
        self._map[from_row][from_col].append(creature.get_drop())
    return place_result

  def move_creature(self, map_creature, move):
    return self.move(map_creature, map_creature.row, map_creature.col, move)

  def print(self, map_thing=None, flashlight=5):
    map_str = ''
    for r in range(self.height):
      if map_thing != None and (r < map_thing.row - flashlight or r > map_thing.row + flashlight):
        map_str += self._fog.t_char * self.width
      else:
        for c in range(self.width):
          if map_thing != None and (c < map_thing.col - flashlight or c > map_thing.col + flashlight):
            map_str += self._fog.t_char
          else:
            map_str += self._map[r][c][-1].t_char
      map_str += '\n'
    return map_str

  def __len__(self):
    return self.width * self.height
  
  def __repr__(self):
    return self.print()

class Game:
  def __init__(self, size=(10, 10), level=0, player_controller=None, smart_controller=None):
    self.level = level
    self.map = Map(*size)
    self.resets = 4
    self.goal = Map_thing('@', 1)
    self.player = Creature('#', 2)
    self.player_controller = SimpleController(self.player, self.goal) if player_controller == None else player_controller
    # self.player.setController(self.player_controller)
    self.player_controller.setCreature(self.player)
    self.controllers = [self.player_controller]
    self.vampires = []
    self.other_creatures = []
    self.init_map()


  def init_map(self):
    self.turns_taken = 0
    self.map.re_init_map()

    vamp_len = len(self.vampires)

    def no_print(message):
      pass
    
    while len(self.vampires) < self.level:
      vamp = Creature('V', 5, num_moves=2)
      control = SimpleController(vamp, self.player, looker=no_print, responder=no_print)
      self.controllers.append(control)
      self.vampires.append(vamp)

    wrappings = Map_thing('W', 7)
    for level in range(vamp_len, self.level):
      if level != 0:
        if level % 5 == 0:
          mummy = Creature('M', 8, drop_thing=wrappings)
          control = SimpleController(mummy, self.player, looker=no_print, responder=no_print)
          self.controllers.append(control)
          self.other_creatures.append(mummy)
        if level % 10 == 0:
          ghost = Creature('G', 11, wait_moves=5)
          control = SimpleController(ghost, self.player, looker=no_print, responder=no_print)
          self.controllers.append(control)
          self.other_creatures.append(ghost)

    for other in self.other_creatures:
      self.map.rand_place(other)
    for v in self.vampires:
      self.map.rand_place(v)
    for c in self.other_creatures:
      self.map.rand_place(c)  
    
    self.map.rand_place(self.player)
    self.map.rand_place(self.goal)

  def check_win(self):
    win = self.player.row == self.goal.row and self.player.col == self.goal.col
    if win:
      self.level += 1
      self.turns_taken = 0
    return win

  def check_loss(self):
    for c in self.vampires:
      if self.player.row == c.row and self.player.col == c.col:
        return True
    for other in self.other_creatures:
      if self.player.row == other.row and self.player.col == other.col:
        return True
    return False

  def broadcastMessage(self, message):
    for controller in self.controllers:
      controller.sendResponse(message)

  def run(self, turns=None):
    turns = len(self.map) * 2 if turns == None else turns
    player_wins = False
    while self.turns_taken < turns:
      self.turns_taken += 1
      for controller in self.controllers:
        map_info = self.map.print(map_thing=controller.getCreature())
        controller.look(map_info)
        if controller.canMove():
          move = controller.getMove()
          if move == 'reset' and self.resets > 0:
            self.resets -= 1
            controller.sendResponse(str(self.resets))
            self.init_map()
            break
          if move == 'quit':
            self.turns_taken = turns + 1
            self.broadcastMessage('Quitting')
            return 'Quitting'
          move_result = self.map.move_creature(controller.getCreature(), move)
          player_wins = self.check_win()
          if player_wins:
            self.broadcastMessage('Player leveled!')
            self.init_map()
          if self.check_loss():
            self.broadcastMessage('Player lost!')
            return 'Loss'
        else:
          controller.sendResponse("Can't move")

    self.broadcastMessage('Too long')
    return 'Too Long'





if __name__ == "__main__":

  g = Game(level=11, size=(40, 40), player_controller=ConsoleController())
  g.run()
  print(g.map)

  



****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
****************************************
*******************************     █  █
*******************************  █ █  ██
****************